In [3]:
using Distributed
@everywhere begin 
    using ParallelTemperingMonteCarlo
    using DelimitedFiles
    using BenchmarkTools
end

In [2]:
using Random,Plots

#set random seed - for reproducibility
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 100000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=12.0*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)



Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[], Float64[])

In [3]:
ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=true)
#plot(temp.t_grid,results.heat_cap)

# data = [results.en_histogram[i] for i in 1:n_traj]
# plot(data)

Total number of moves per MC cycle: 13



equilibration done


MC loop done.


[0.0003274332206663901, 0.00034097451808384846, 0.00035244948549771713, 0.00038565890500638536, 0.0003980335513622917, 0.00041254941004158604, 0.0004387579945671251, 0.00046105266182069345, 0.00048549186855110376, 0.0005021570901080064, 0.0005451170846650248, 0.0005775643450812362, 0.0006458749223787228, 0.0006770807764240569, 0.00071711013573714, 0.0008368237946829951, 0.0009831458227094459, 0.0011601596513707078, 0.0013070362090407607, 0.0017146656421000697, 0.0019621319526630914, 0.0030365686353476936, 0.0036432380891126223, 0.003581044352305293, 0.003610675334464941, 0.0032390163705791784, 0.0028215089701653243, 0.0028292964927889848, 0.003584854417093773, 0.003757680935178654, 0.004212266642843023, 0.004559761171251476]
done


Testing better mc_cycles

In [24]:
@everywhere function parallelmc_cycle!(mc_states, move_strat, mc_params, pot, ensemble, n_steps, a, v, r)
    #perform one MC cycle
    Threads.@threads for i_traj = 1:mc_params.n_traj
        for i_step = 1:n_steps
            #mc_states[i_traj] = mc_step!(type_moves[ran][2], type_moves[ran][1], mc_states[i_traj], ran, pot, ensemble)
            @inbounds mc_states[i_traj] = mc_step!(mc_states[i_traj], pot, ensemble, a, v, r)
        end
        #push!(mc_states[i_traj].ham, mc_states[i_traj].en_tot) #to build up ham vector of sampled energies
    end
    #parallel tempering
    if rand() < 0.1 #attempt to exchange trajectories
        n_exc = rand(1:mc_params.n_traj-1)
        mc_states[n_exc].count_exc[1] += 1
        mc_states[n_exc+1].count_exc[1] += 1
        exc_acc = exc_acceptance(mc_states[n_exc].beta, mc_states[n_exc+1].beta, mc_states[n_exc].en_tot,  mc_states[n_exc+1].en_tot)
        if exc_acc > rand()
            mc_states[n_exc].count_exc[2] += 1
            mc_states[n_exc+1].count_exc[2] += 1
            mc_states[n_exc], mc_states[n_exc+1] = exc_trajectories!(mc_states[n_exc], mc_states[n_exc+1])
        end
    end
    return mc_states
end

@everywhere function paralleltest(var)
    if var == -1
        for i in 1:100
            mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    elseif var == 1
        for i in 1:100
            parallelmc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    end
end


In [5]:
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time1 = @benchmark paralleltest(-1)

mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time3 = @benchmark paralleltest(1)

BenchmarkTools.Trial: 2788 samples with 1 evaluation.
 Range (min … max):  909.050 μs … 63.657 ms  ┊ GC (min … max):  0.00% … 97.53%
 Time  (median):       1.513 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.784 ms ±  2.724 ms  ┊ GC (mean ± σ):  15.71% ± 10.07%

  ▃█▂                                                           
  ███▄▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▂▂ ▂
  909 μs          Histogram: frequency by time         22.2 ms <

 Memory estimate: 1.70 MiB, allocs estimate: 17749.

In [6]:
println("no parallel = ",time1," parallelcycle = ",time3)

no parallel = Trial

(1.139 ms) parallelcycle = Trial(909.050 μs)


In [7]:
@everywhere function paralleltest(var)
    if var == -1
        for i in 1:1000
            mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    elseif var == 1
        for i in 1:1000
            parallelmc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    end
end

mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time1 = @benchmark paralleltest(-1)

mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time3 = @benchmark paralleltest(1)

println("no parallel = ",time1," parallelcycle = ",time3)

no parallel = Trial(12.223 ms) parallelcycle = Trial(10.159 ms)


In [16]:
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
@benchmark begin 
    for i in 1:1000
        parallelmc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
    end
end


BenchmarkTools.Trial: 273 samples with 1 evaluation.
 Range (min … max):   9.882 ms … 47.833 ms  ┊ GC (min … max):  0.00% … 54.76%
 Time  (median):     16.596 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   18.322 ms ±  8.164 ms  ┊ GC (mean ± σ):  15.35% ± 19.96%

             █▄                                                
  ▃▄▄▇▅▃▃▃▄▃▆██▇▃▃▃▂▂▁▃▂▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▂▃▂▂▂▁▁▂▂▂▂▂▃▄▂▁▂ ▂
  9.88 ms         Histogram: frequency by time        43.6 ms <

 Memory estimate: 16.98 MiB, allocs estimate: 178310.

In [20]:
@benchmark begin
    ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=true)
end

Total number of moves per MC cycle: 13



equilibration done


MC loop done.
[0.0003274332206663901, 0.00034097451808384846, 0.00035244948549771713, 0.00038565890500638536, 0.0003980335513622917, 0.00041254941004158604, 0.0004387579945671251, 0.00046105266182069345, 0.00048549186855110376, 0.0005021570901080064, 0.0005451170846650248, 0.0005775643450812362, 0.0006458749223787228, 0.0006770807764240569, 0.00071711013573714, 0.0008368237946829951, 0.0009831458227094459, 0.0011601596513707078, 0.0013070362090407607, 0.0017146656421000697, 0.0019621319526630914, 0.0030365686353476936, 0.0036432380891126223, 0.003581044352305293, 0.003610675334464941, 0.0032390163705791784, 0.0028215089701653243, 0.0028292964927889848, 0.003584854417093773, 0.003757680935178654, 0.004212266642843023, 0.004559761171251476]
done
Total number of moves per MC cycle: 13

equilibration done


MC loop done.


[-3.626054009716755, -3.4767364387711974, -3.3335852854731485, -3.1913119524479177, -3.0609508279365523, -2.929262012745028, -2.80396322548002, -2.68362798068806, -2.5645866137669118, -2.451813314825442, -2.3459789171327254, -2.2418599772617656, -2.1365739853181696, -2.0402277808832627, -1.9425366807239308, -1.8488257813857005, -1.7579869401528843, -1.6635693401784761, -1.573272692441443, -1.4688130215294692, -1.3838083758570572, -1.2673403795741625, -1.157128403465078, -1.0334861083804563, -0.9331932567256628, -0.8429358337762167, -0.7583379434458081, -0.6955050500668383, -0.6314473348158312, -0.556941565544621, -0.49089888279758415, -0.4387188569491709]
done
Total number of moves per MC cycle: 

...


[-10.883889797317497, -10.431804745433759, -9.99574737212787, -9.577542992257698, -9.182177879418681, -8.787921033113072, -8.415032597157204, -8.052503609832897, -7.695688582446834, -7.356640070754599, -7.034295159245451, -6.7270310455825335, -6.415890542780838, -6.122637725728883, -5.8341208863918785, -5.551259689315166, -5.280678871729084, -5.005277442232245, -4.730671735025874, -4.426748050972376, -4.162053288087249, -3.8118201697941774, -3.489070624361203, -3.139951779223422, -2.837044815519595, -2.5449006568366954, -2.288194524658741, -2.0877040642525357, -1.883551143956719, -1.6618830580577397, -1.4891663632854393, -1.337553462151169]
done
Total number of moves per MC cycle: 

13

equilibration done


MC loop done.


[-21.76448542527407, -20.864765358539646, -19.99269636891073, -19.15493697281766, -18.357619616682374, -17.575687160125995, -16.833466754240042, -16.104230690610883, -15.391133972223683, -14.718280757024107, -14.07310279391676, -13.44995067204151, -12.831050367633143, -12.247488097722556, -11.67487636182148, -11.106036215574068, -10.563048451410294, -10.02101816405747, -9.470832147413903, -8.893377337680015, -8.32579421426003, -7.638394934995636, -6.987729123817622, -6.243411981760421, -5.634190141705224, -5.075123637753679, -4.564592035963244, -4.165291092318767, -3.7628272502477613, -3.3246711371716122, -2.9789477958494084, -2.650587310617719]
done


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 35.026 s (3.04% GC) to evaluate,
 with a memory estimate of 25.64 GiB, over 183505175 allocations.

In [30]:


@everywhere function parallelptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i ;delta_en=0. )
    #perform one MC cycle
    Threads.@threads for i_traj = 1:mc_params.n_traj
        for i_step = 1:n_steps
            #mc_states[i_traj] = mc_step!(type_moves[ran][2], type_moves[ran][1], mc_states[i_traj], ran, pot, ensemble)
            @inbounds mc_states[i_traj] = mc_step!(mc_states[i_traj], pot, ensemble, a, v, r)
        end
        #push!(mc_states[i_traj].ham, mc_states[i_traj].en_tot) #to build up ham vector of sampled energies
    end
    #parallel tempering
    if rand() < 0.1 #attempt to exchange trajectories
        n_exc = rand(1:mc_params.n_traj-1)
        mc_states[n_exc].count_exc[1] += 1
        mc_states[n_exc+1].count_exc[1] += 1
        exc_acc = exc_acceptance(mc_states[n_exc].beta, mc_states[n_exc+1].beta, mc_states[n_exc].en_tot,  mc_states[n_exc+1].en_tot)
        if exc_acc > rand()
            mc_states[n_exc].count_exc[2] += 1
            mc_states[n_exc+1].count_exc[2] += 1
            mc_states[n_exc], mc_states[n_exc+1] = exc_trajectories!(mc_states[n_exc], mc_states[n_exc+1])
        end
    end
    sampling_step!(mc_params,mc_states,i,save_ham)

    if save_ham == false
        updatehistogram!(mc_params,mc_states,results,delta_en,fullham=save_ham)
    end

    #step adjustment
    if rem(i, mc_params.n_adjust) == 0
        for i_traj = 1:mc_params.n_traj
            update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
        end 
    end

    if save == true
        if rem(i,1000) == 0
            save_states(mc_params,mc_states,i)
            if save_ham == false
                save_results(results)
            end
        end
    end
    
end

@everywhere function ptmc_threaded_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham::Bool = true, save::Bool=true, restart::Bool=false,restartindex::Int64=0)
    #restart isn't compatible with saving the hamiltonian at the moment

    if restart == true
        save_ham = false
    end
    
    if save_ham == false
        if restart == false
            #If we do not save the hamiltonian we still need to store the E, E**2 terms at each cycle
            for i_traj = 1:mc_params.n_traj
                push!(mc_states[i_traj].ham, 0)
                push!(mc_states[i_traj].ham, 0)
            end
        end
    end

    a = atom_move_frequency(move_strat)
    v = vol_move_frequency(move_strat)
    r = rot_move_frequency(move_strat)
    #number of MC steps per MC cycle
    n_steps = a + v + r

    println("Total number of moves per MC cycle: ", n_steps)
    println()

    
    #equilibration cycle
    if restart == false
        #this initialises the max and min energies for histograms
        if save_ham == false
            ebounds = [100. , -100.] #emin,emax
        end
        
         for i = 1:mc_params.eq_cycles
            @inbounds mc_states = mc_cycle!(mc_states, move_strat, mc_params, pot, ensemble, n_steps, a, v, r)
            #verbose way to save the highest and lowest energies

            if save_ham == false
                for i_traj = 1:mc_params.n_traj
                    if mc_states[i_traj].en_tot < ebounds[1]
                        ebounds[1] = mc_states[i_traj].en_tot
                    end

                    if mc_states[i_traj].en_tot > ebounds[2]
                        ebounds[2] = mc_states[i_traj].en_tot
                    end
                end
            end
            #update stepsizes
            if rem(i, mc_params.n_adjust) == 0
                for i_traj = 1:mc_params.n_traj
                    update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
                end 
            end
        end
    #re-set counter variables to zero
        for i_traj = 1:mc_params.n_traj
            mc_states[i_traj].count_atom = [0, 0]
            mc_states[i_traj].count_vol = [0, 0]
            mc_states[i_traj].count_rot = [0, 0]
            mc_states[i_traj].count_exc = [0, 0]
        end
        #initialise histogram for non-saving hamiltonian 
        if save_ham == false
            delta_en = initialise_histograms!(mc_params,mc_states,results, full_ham=false,e_bounds=ebounds)
        end

        println("equilibration done")
        

        if save == true
            save_states(mc_params,mc_states,0)
        end
    end


    #main MC loop

    if restart == false

        Threads.@threads for i = 1:mc_params.mc_cycles
            if save_ham == false
                parallelptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i;delta_en=delta_en)
            else
                parallelptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i)
            end
            
        end

    else #if restarting

        for i = restartindex:mc_params.mc_cycles
            if save_ham == false
                parallelptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i;delta_en=delta_en)
            else
                parallelptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i)
            end
            
        end 
    end
    println("MC loop done.")
    #Evaluation
    #average energy
    n_sample = mc_params.mc_cycles / mc_params.mc_sample

    if save_ham == true
        en_avg = [sum(mc_states[i_traj].ham) / n_sample for i_traj in 1:mc_params.n_traj] #floor(mc_cycles/mc_sample)
        en2_avg = [sum(mc_states[i_traj].ham .* mc_states[i_traj].ham) / n_sample for i_traj in 1:mc_params.n_traj]
    else
        en_avg = [mc_states[i_traj].ham[1] / n_sample  for i_traj in 1:mc_params.n_traj]
        en2_avg = [mc_states[i_traj].ham[2] / n_sample  for i_traj in 1:mc_params.n_traj]
    end


    results.en_avg = en_avg

    #heat capacity
    results.heat_cap = [(en2_avg[i]-en_avg[i]^2) * mc_states[i].beta for i in 1:mc_params.n_traj]

    #acceptance statistics
    results.count_stat_atom = [mc_states[i_traj].count_atom[1] / (mc_params.n_atoms * mc_params.mc_cycles) for i_traj in 1:mc_params.n_traj]
    results.count_stat_exc = [mc_states[i_traj].count_exc[2] / mc_states[i_traj].count_exc[1] for i_traj in 1:mc_params.n_traj]

    println(results.heat_cap)

    #energy histograms
    if save_ham == true
        # T = typeof(mc_states[1].ham[1])
        delta_en= initialise_histograms!(mc_params,mc_states,results)
        updatehistogram!(mc_params,mc_states,results,delta_en)
    
    end
    #     for i_traj in 1:mc_params.n_traj
    #         hist = zeros(results.n_bin)#EnHist(results.n_bin, global_en_min, global_en_max)
    #         for en in mc_states[i_traj].ham
    #             index = floor(Int,(en - global_en_min) / delta_en) + 1
    #             hist[index] += 1
    #         end
    #         push!(results.en_histogram, hist)
    #     end
    # end

    #TO DO
    # volume (NPT ensemble),rot moves ...
    # move boundary condition from config to mc_params?
    # rdfs

    println("done")
    return 
end

In [31]:
@benchmark begin
    ptmc_threaded_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=true)
end

Total number of moves per MC cycle: 13



equilibration done


MC loop done.


[0.061310948024631384, 0.05254421136467583, 0.05117483264528824, 0.03424563097504432, 0.03329167264772939, 0.03051078085860182, 0.0333820625391444, 0.029704063666196965, 0.029093709597504548, 0.029534668585531554, 0.02910348574183897, 0.02997704560100222, 0.028319971699097307, 0.025655663207952713, 0.028614644085945794, 0.027514677390198097, 0.02582355504543463, 0.024593801039249828, 0.028950833299674357, 0.024281358155356456, 0.02899641049697663, 0.024996145516832345, 0.022603881464319228, 0.031145057877807712, 0.02847354053083187, 0.02319416153751276, 0.02499139053970299, 0.020797940317747805, 0.023982283708294732, 0.020002048167339095, 0.015209506927165401, 0.01570220156523748]
done
Total number of moves per MC cycle: 13

equilibration done

MC loop done.
[-6.507853463499358, -3.923346325308616, -3.0099081570102957, -2.547312458689133, -2.246809273536758, -2.0644614526424028, -1.855855732347191, -1.6976965748483122, -1.5280869385572096, -1.3701273602254969, -1.2728261076222924, -1.150757391819256, -1.0583974752924143, -0.9539783543350052, -0.8948737815792366, -0.8544335887395647, -0.7860402745023489, -0.7207290131204475, -0.6665077664864484, -0.5957145190717847, -0.5391312600275582, -0.4874377462874685, -0.43917405876095944, -0.3823304943164629, -0.30622781552371126, -0.2653503864785152, -0.22216687489788148, -0.14867636512282517, -0.08653326953225761, -0.053457196210416565, -0.030525886521364818, -0.0065329084128453235]
done
Total number of moves per MC cycle: 

13

equilibration done

MC loop done.
[-27.273613335513417, -17.17262638007241, -13.898303704050008, -12.112589066539776, -10.880991923129416, -9.654344685063528, -8.81184824396772, -8.098493210823657, -7.434467623498053, -6.810796807080801, -6.337017815936543, -5.8538404846511725, -5.424698676282123, -4.993923844669324, -4.660374968350754, -4.425606450364899, -4.139594290252852, -3.832837900190097, -3.5904762993188433, -3.3048234428887264, -3.0404787302328624, -2.7804121247647755, -2.5560335141528867, -2.315776346565898, -2.0176586366937252, -1.7788458928319164, -1.5377868353793798, -1.1751786847029837, -0.8470839057516436, -0.6056771830070735, -0.4301443975751873, -0.296269864910662]
done
Total number of moves per MC cycle: 

13

equilibration done

MC loop done.
[-64.79626347959585, -45.50553819508016, -38.46123048667198, -33.83652979822183, -30.773361875805083, -27.45138794133506, -24.983092038684116, -23.00885986688557, -21.295219571329998, -19.64224438811465, -18.349974027962272, -17.055747934055624, -15.880766208950059, -14.696493955741913, -13.687496648273449, -12.802420800855627, -11.88012439413038, -11.00337522557386, -10.13846521857316, -9.427411961970796, -8.725548553934415, -8.00524341319705, -7.437828348794091, -6.846033071185029, -6.15556901898636, -5.534078232923297, -4.931184655279797, -4.049189370639596, -3.2278567134282277, -2.574791239497234, -2.011766136748435, -1.6176282562694009]
done


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 25.045 s (13.24% GC) to evaluate,
 with a memory estimate of 26.37 GiB, over 189922426 allocations.

In [16]:
using Random
@everywhere import ..MCRun: initialise_histograms!,updatehistogram!,update_max_stepsize!,sampling_step!,save_results,save_states
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 10000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

runnerdir = "/home/ghun245/RuNNer-master/Brass_potential/"
atomtype="Cu"
pot = AbstractMLPotential(runnerdir,atomtype)

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
ico_13  = [[-0.0000000049,       -0.0000000044,       -0.0000000033],
[-0.0000007312,       -0.0000000014,        0.6554619119],
 [0.1811648930,      -0.5575692094,        0.2931316798],
[-0.4742970242,       -0.3445967289,        0.2931309525],
[-0.4742970303,        0.3445967144,        0.2931309494],
 [0.1811648830,        0.5575692066,        0.2931316748],
 [0.5862626299,        0.0000000022,        0.2931321262],
[-0.1811648928,       -0.5575692153,       -0.2931316813],
[-0.5862626397,       -0.0000000109,       -0.2931321327],
[-0.1811649028,        0.5575692007,       -0.2931316863],
 [0.4742970144,        0.3445967202,       -0.2931309590],
 [0.4742970205,       -0.3445967231,       -0.2931309559],
 [0.0000007214,       -0.0000000073,       -0.6554619185]]

#convert to Bohr
nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu13 = copperconstant*ico_13*1.5
AtoBohr = 1.8897259886


#boundary conditions 
bc_cu13 = SphericalBC(radius=12.0*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_cu13, bc_cu13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

cd("/home/ghun245/ParallelTemperingMonteCarlo.jl/scripts")



In [24]:
function init_parallel_RuNNer(pot::AbstractMLPotential; n_threads = Threads.nthreads())
    if isdir("pots") == true
        rm("pots",recursive = true)
    end
    potential_directory = mkdir("$(pwd())/pots")
    potential_vector = []
    for i = 1:n_threads
        # n_threads potentials
        cp(pot.dir,"$(potential_directory)/potential$i")
        pot_i = AbstractMLPotential("$(potential_directory)/potential$i/",pot.atomtype)
        push!(potential_vector,pot_i)
    end
        
    return potential_vector        
end

function parallel_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)
    parallel_states = []

    pot_vector = init_parallel_RuNNer(pot) #initialise state and potentials

    n_threads = Threads.nthreads()
    sample_index = Int64(floor(mc_params.eq_cycles / n_threads)) #number of eq cycles per thread

    a = atom_move_frequency(move_strat)
    v = vol_move_frequency(move_strat)
    r = rot_move_frequency(move_strat)
    #number of MC steps per MC cycle
    n_steps = a + v + r

    println("Total number of moves per MC cycle: ", n_steps)
    println()

    ebounds = [100. , -100.]

    for i_thread = 1:n_threads
        for i_eq = 1:sample_index
            
            i = i_thread*sample_index + i_eq

            mc_states = mc_cycle!(mc_states, move_strat, mc_params, pot_vector[i_thread], ensemble, n_steps, a, v, r)#mc cycle
            
            for i_traj = 1:mc_params.n_traj#check energy bounds
                if mc_states[i_traj].en_tot < ebounds[1]
                    ebounds[1] = mc_states[i_traj].en_tot
                end

                if mc_states[i_traj].en_tot > ebounds[2]
                    ebounds[2] = mc_states[i_traj].en_tot
                end

            end

            if rem(i, mc_params.n_adjust) == 0 #adjust stepsize
                for i_traj = 1:mc_params.n_traj
                    update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
                end 
            end

        end
        
        states_vec = [MCState(mc_states[i_traj].temp,mc_states[i_traj].beta,mc_states[i_traj].config,pot_vector[i_thread];max_displ = mc_states[i_traj].max_displ ) for i_traj in 1:mc_params.n_traj] #initialise a new mc_states vector based on current state

        push!(parallel_states,states_vec) #add to vector of parallel states


    end
    
    delta_en = initialise_histograms!(mc_params,mc_states,results, full_ham=false,e_bounds=ebounds) #start histogram

    println("equilibration done")
    
    return parallel_states,pot_vector,a,v,r,delta_en,n_threads

end

parallel_equilibration (generic function with 1 method)

In [25]:
parallel_states,pot_vector,a,v,r,delta_en,n_threads = parallel_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)

(Any[MCState{Float64, 13, SphericalBC{Float64}}[MCState{Float64, 13, SphericalBC{Float64}}(5.0, 63154.997052619445, Config{13, SphericalBC{Float64}, Float64}(StaticArraysCore.SVector{3, Float64}[[0.022904457986679054, -0.0996853659131234, 0.03890085876681727], [0.03094994420275727, -0.08143023744612053, 7.203069036053088], [2.1075896259118254, -6.172002403350006, 3.134109792591704], [-5.197810721142771, -3.8681704949346636, 3.1666805106186504], [-5.170712010915881, 3.670317000036486, 3.2488813954562787], [1.9075186065449614, 6.1042186649885055, 3.107714514542579], [6.51510239957335, -0.011545667969870502, 3.227239782277621], [-1.9974924948089847, -6.141600065262699, -3.263056624550947], [-6.349706511627254, 0.08924419837957598, -3.264998046772701], [-1.894997678874064, 6.131675746996192, -3.227175650965221], [5.2426787527878185, 3.6517817101313526, -3.353880325204192], [4.917741245940232, -3.8588620763453467, -3.26174090766196], [-0.06672752437289456, -0.019540800542388957, -7.23380484

In [84]:
@everywhere function threadexchange(parallel_states,n_threads,idx)
    if rand() < 0.2  #20% change per trajectory of an attempted exchange
        thread_id = rand(1:n_threads-1) #which threads are talking

        exc_acc = exc_acceptance(parallel_states[thread_id][idx].beta,parallel_states[thread_id+1][idx].beta,parallel_states[thread_id][idx].en_tot,parallel_states[thread_id+1][idx].en_tot) #calc acceptance
        
        if exc_acc > rand() #if exchange is likely
            exc_trajectories!(parallel_states[thread_id][idx] ,parallel_states[thread_id+1][idx] )#swap
        end
     end
    
end



In [92]:

@everywhere function pptmc_loop!(parallel_states,results,move_strat,mc_params,pot_vector,ensemble,n_steps,a,v,r,save_ham,save,i,delta_en,n_threads)
    @sync begin
        Threads.@threads for i_thread = 1:n_threads
        #run ~1000 steps then attempt exchanges
            for i = 1:1000
                ptmc_cycle!(parallel_states[i_thread],results,move_strat,mc_params,pot_vector[i_thread],ensemble,n_steps,a,v,r, save_ham,save,i,delta_en)
            end

        end
    end
    #attempt exchanges
    @sync begin
        for idx = 1:mc_params.n_traj
            threadexchange(parallel_states,n_threads,idx)
        end
    end

end

@everywhere function pptmc_run!(mc_states,move_strat,mc_params,pot,ensemble,results; save_ham = false,save = false)
    #we can initialise everything in bulk. 
    parallel_states,pot_vector,a,v,r,delta_en,n_threads = parallel_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)
    println("equilibration complete")

    num_loops = Int(floor(mc_params.mc_cycles/(1000*n_threads) ))
    counter = 0

    for i_loop = 1:num_loops
        pptmc_loop!(parallel_states,results,move_strat,mc_params,pot_vector,ensemble,n_steps,a,v,r,save_ham,save,i,delta_en,n_threads)

        counter += n_threads*1000
    end
    println("MC loop done")

    rm("pots",recursive = true)

    #post processing
    en_tot_avg = zeros(mc_params.n_traj)
    en2_tot_avg = zeros(mc_params.n_traj)
    count_stat_atom_tot = zeros(mc_params.n_traj)
    count_stat_exc_tot = zeros(mc_params.n_traj)

    for statevec in parallel_states
        en_avg = [statevec[i_traj].ham[1] / n_sample  for i_traj in 1:mc_params.n_traj]

        en_tot_avg += en_avg

        en2_avg = [statevec[i_traj].ham[2] / n_sample  for i_traj in 1:mc_params.n_traj]
        en2_tot_avg += en2_avg

        count_stat_atom = [mc_states[i_traj].count_atom[1] / (mc_params.n_atoms * mc_params.mc_cycles) for i_traj in 1:mc_params.n_traj]
        count_stat_atom_tot += count_stat_atom
        count_stat_exc = [mc_states[i_traj].count_exc[2] / mc_states[i_traj].count_exc[1] for i_traj in 1:mc_params.n_traj]
        count_stat_exc_tot += count_stat_exc


    end
    results.en_avg = en_avg

    results.count_stat_atom,results.count_stat_exc = count_stat_atom_tot,count_stat_exc_tot

    results.heat_cap = [(en2_avg[i]-en_avg[i]^2) * mc_states[i].beta for i in 1:mc_params.n_traj]
    println(results.heat_cap)
    println("done")
    return

end


I want to benchmark the exchanges between threads -- this was done and it is slower to thread the exchange step

In [99]:
rm("pots",recursive=true)

In [98]:
cd("/home/ghun245/ParallelTemperingMonteCarlo.jl/scripts/")
pwd()

"/home/ghun245/ParallelTemperingMonteCarlo.jl/scripts"